# Setup

Imports:

In [2]:
from dataclasses import dataclass
import random

### Defining classes

* **Node** <br>
  Every person in the simulation is a node.
  * `neighbors`: a list of all of the Node's neighbors, which are also Nodes.
  * `degree()`: returns the number of neighbors the Node has -- its "degree".

* **Graph** <br>
  A class for all of the graphs. <br>
  Contains a list of the Nodes in the graph. <br>
  For example in this case we will have one "Icelandic" Graph and one "English" Graph.
  * `nodes`: a list of all of the Nodes in this Graph.
  * `create_edge`: gets two nodes. If they are both in the graph and are not already neighbors, each one is added to the other's list of neighbors, thus creating an edge between them.

In [3]:
class Node:
    def __init__(self):
        self.neighbors: list["Node"] = []
        self.friend_group: float = random.random()
        self.frequency: float = 0
        
    def __repr__(self):
        return f"Node[{len(self.neighbors)}]"
    
    def degree(self) -> int:
        return len(self.neighbors)
    

class Graph:
    def __init__(self):
        self.nodes: list[Node] = []
    
    def __str__(self) -> str:
        return str(self.nodes)

    def create_edge(self, node1: Node, node2: Node) -> None:
        if node1 not in self.nodes or node2 not in self.nodes:
            raise ValueError("Node not in Graph")
        if node1 in node2.neighbors or node2 in node1.neighbors:
            raise ValueError("Edge already exists")
        node1.neighbors.append(node2)
        node2.neighbors.append(node1)


### Creating Graph

This function gets the following parameters:
* `node_num`: the number of Nodes that are going to be in the Graph.
* `edge_prob`: the probability two random nodes would have an edge between them.
And returns a Graph.

`friendliness`: The difference between the friend group of `g.nodes[i]` and `g.nodes[j]`, i.e. the two nodes we want to connect.

Every iteration, the function creates a node, then goes over all of the nodes that were already created (this way the same pair of nodes will not occur twice). After going through the probability check, it creates an edge between the two nodes using `create_edge`.

**TODO**: not sure how to confirm that the friendliness probability check works

**TODO**: the friendliness thing makes it much more probable to have an edge between two nodes (because it is added to `edge_prob`). That means if `edge_prob` is 0.5 and `node_num` is 100, the average number of neighbors for each node will be around 78, while before the friendliness addition it would have been 50. Not really sure if that's a problem, and if it is how to solve it.



In [4]:
def create_graph_simple(node_num: int, edge_prob: float) -> Graph:
    g = Graph()
    for i in range(node_num):
        g.nodes.append(Node())
        for j in range(len(g.nodes)-1):
            if random.random() < edge_prob:
                g.create_edge(g.nodes[i], g.nodes[j])
    return g

In [5]:
def create_graph_w_friend(node_num: int, edge_prob: float) -> Graph:
    g = Graph()
    for i in range(node_num):
        g.nodes.append(Node())
        for j in range(len(g.nodes)-1):
            friendliness = abs(g.nodes[i].friend_group - g.nodes[j].friend_group)
            if random.random() < edge_prob + friendliness:
                g.create_edge(g.nodes[i], g.nodes[j])
    return g

Testing:

In [6]:
NUM_NODES = 100
a = create_graph_simple(NUM_NODES, 0.5)
total = 0
for n in a.nodes:
    total += n.degree()
print(a)
print(total/NUM_NODES)

[Node[51], Node[53], Node[44], Node[49], Node[59], Node[48], Node[57], Node[50], Node[53], Node[49], Node[52], Node[44], Node[37], Node[54], Node[56], Node[54], Node[45], Node[45], Node[51], Node[52], Node[50], Node[54], Node[42], Node[58], Node[47], Node[51], Node[42], Node[46], Node[50], Node[47], Node[50], Node[52], Node[54], Node[51], Node[56], Node[49], Node[59], Node[52], Node[52], Node[51], Node[54], Node[49], Node[48], Node[51], Node[59], Node[46], Node[49], Node[47], Node[50], Node[55], Node[54], Node[41], Node[48], Node[56], Node[55], Node[45], Node[48], Node[42], Node[49], Node[45], Node[50], Node[51], Node[51], Node[45], Node[43], Node[50], Node[45], Node[41], Node[44], Node[61], Node[49], Node[47], Node[49], Node[51], Node[46], Node[48], Node[55], Node[61], Node[55], Node[42], Node[44], Node[53], Node[35], Node[56], Node[53], Node[44], Node[49], Node[50], Node[50], Node[48], Node[44], Node[54], Node[55], Node[43], Node[52], Node[47], Node[49], Node[47], Node[49], Node[51]]

### Timestep Function


TODO: here should be a function that will be run every timestep. It will include two for loops that go over all nodes in the graph: in the first one it will put into a dictionary the position of the current node in the list of nodes, which will lead to its new frequency. In the second one it will go over the dictionary and update the frequencies.

In [7]:
def step_rule_follow_random_neighbor(node: Node) -> float:
    return random.choice(node.neighbors).frequency

In [9]:
def run_timestep(g: Graph) -> None:
    # Goes through all of the nodes in the graph and adds it to a dictionary, pointing at its soon-to-be frequency.
    # Then goes through all of the nodes again.
    # Has to go through all of them twice so that the function step_rule will consider only the old frequencies and not the new ones. 
    d: dict[Node, float] = {}
    for node in g.nodes:
        d[node] = step_rule_follow_random_neighbor(node)
    for node, freq in d.items():
        node.frequency = freq

# Main